In [19]:
import os
import numpy as np
import pandas as pd
from atusfunclib import load_data
from sqlalchemy import create_engine
import yaml

In [20]:
PATH = "/home/vagrant/capstoneproject/TDICapstoneProject/data"

In [21]:
with open('config', 'r') as f:
    config = yaml.load(f)
    SQL_PASSWORD = config['sql']['pass']
    SQL_USERNAME = config['sql']['user']

In [3]:
# Import all data
data_import = load_data(loc='data')

In [4]:
# Unpack individual dataframes
df, dfactcodes, dfeducodes, dfinccodes, dfagecodes, \
dfempcodes, dfindcodes, dfraccodes, dfloccodes, dfwhocodes, \
dfdemocodes = data_import

In [5]:
# Dtype convertion
dtypedict = {'int64': 'BIGINT', 'float64': 'REAL', 'category': 'TEXT'}

In [13]:
# Code tables
codedfs = {'actcodes': dfactcodes, 'educodes': dfeducodes, 'inccodes': dfinccodes,
           'agecodes': dfagecodes, 'empcodes': dfempcodes, 'indcodes': dfindcodes,
           'raccodes': dfraccodes, 'loccodes': dfloccodes, 'whocodes': dfwhocodes,
           'democodes': dfdemocodes}
# Data tables
datadfs = {'actimesw3': df.filter(regex=r'TUCASEID|t\d{6}_W'),
           'actimes3':  df.filter(regex=r'TUCASEID|t\d{6}$'),
           'actimesw2': df.filter(regex=r'TUCASEID|t\d{4}_W'),
           'actimes2':  df.filter(regex=r'TUCASEID|t\d{4}$'),
           'actimesw1': df.filter(regex=r'TUCASEID|t\d{2}_W'),
           'actimes1':  df.filter(regex=r'TUCASEID|t\d{2}$'),
           'demow':     df.filter(regex=r'TUCASEID|^[A-Z]+_W'),
           'demo':      df.filter(regex=r'TUCASEID|^[A-Z]+[^_W]$')}

# Database name
databasename = 'atusdata'
# Username
username = SQL_USERNAME
# Password
password = SQL_PASSWORD

In [16]:
# Postgres engine for atusdata database
engine = create_engine("postgresql://{}:{}@localhost:5432/{}".format(username, password, databasename))

In [17]:
%%time
# Create code tables (small tables OK to use pandas)
for k in codedfs.keys():
    try:
        codedfs[k].to_sql(k, engine)
    except ValueError:
        print "Table '{}' already exists in '{}'".format(k, databasename)

Table 'raccodes' already exists in 'atusdata'
Table 'loccodes' already exists in 'atusdata'
Table 'actcodes' already exists in 'atusdata'
Table 'indcodes' already exists in 'atusdata'
Table 'empcodes' already exists in 'atusdata'
Table 'whocodes' already exists in 'atusdata'
Table 'agecodes' already exists in 'atusdata'
Table 'inccodes' already exists in 'atusdata'
Table 'educodes' already exists in 'atusdata'
CPU times: user 103 ms, sys: 66 ms, total: 169 ms
Wall time: 287 ms


In [21]:
# Create csv subfiles from data dfs
for k in datadfs.keys():
    datadfs[k].to_csv("data/{}.csv".format(k))

In [33]:
sql_create_statement = """
    CREATE TABLE IF NOT EXISTS {}
        ({});
    """
sql_copy_statement = """
    COPY {} FROM '{}'
        WITH (FORMAT csv, HEADER);
    """
sql_droptable_statement = """
    DROP TABLE {};
"""

In [34]:
# Delete tables
for k in datadfs.keys():
    engine.execute(sql_droptable_statement.format(k))

In [35]:
# Create tables
for k in datadfs.keys():
    
    header = datadfs[k].keys().tolist()
    dtypes = [dtypedict[i.name] for i in datadfs[k].dtypes.tolist()]

    schemestring = "index BIGINT, " + ", ".join([" ".join(i) for i in zip(header, dtypes)])

    engine.execute(sql_create_statement.format(k, schemestring))

In [36]:
%%time
# Copy data from csv files into tables
for k in datadfs.keys():
    
    filenamestring = os.path.join(PATH, "{}.csv".format(k))    
    print filenamestring

    with engine.connect().execution_options(autocommit=True) as con:
        con.execute(sql_copy_statement.format(k, filenamestring))

/home/vagrant/capstoneproject/TDICapstoneProject/data/actimesw2.csv
/home/vagrant/capstoneproject/TDICapstoneProject/data/actimesw3.csv
/home/vagrant/capstoneproject/TDICapstoneProject/data/actimesw1.csv
/home/vagrant/capstoneproject/TDICapstoneProject/data/actimes2.csv
/home/vagrant/capstoneproject/TDICapstoneProject/data/actimes3.csv
/home/vagrant/capstoneproject/TDICapstoneProject/data/actimes1.csv
/home/vagrant/capstoneproject/TDICapstoneProject/data/demo.csv
/home/vagrant/capstoneproject/TDICapstoneProject/data/demow.csv
CPU times: user 60.9 ms, sys: 16.5 ms, total: 77.4 ms
Wall time: 1min 1s
